In [ ]:
import MDAnalysis as mda
import mdtraj as md
import pdbfixer
from openmm.app import PDBFile
import warnings
warnings.filterwarnings('ignore')

# **Upload target PDB file and choose concentrations of probes**

# Replace with the pdb file of interest
pdb_file_name = 'structure.pdb'
#Choose chain(s), comma separated:
ChainID = "A"

# Set relative concentrations for water/probes:
# Water:
W = 0.952
# Phenol:
PHEN = 0.003
# Acetic acid:
ACET = 0.009
# Isopropyl amide:
IPA = 0.009
# Dimethylacetamide:
DMAD = 0.009
# Isopropanol:
IPO = 0.009
# Acetone:
PPN = 0.009

u = mda.Universe(pdb_file_name)
sel = u.select_atoms('protein and segid ' + ChainID.replace(',', ' '))
sel.write('all_atom_selected.pdb')

# fix PDB
fixer = pdbfixer.PDBFixer(filename = 'all_atom_selected.pdb')
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
PDBFile.writeFile(fixer.topology, fixer.positions, open('aa_fixed.pdb', 'w'))

# define the secondary structure
str_ini = md.load('aa_fixed.pdb', top='aa_fixed.pdb')
ss = md.compute_dssp(str_ini, simplified=True)[0]
ss = "'" + ''.join(ss) + "'"

# string with the insane.py keys setting
# the selected probes and their concentrations
sol_str = ' -sol W:' + str(W) + ' -sol PHEN:' + str(PHEN) + \
' -sol ACET:' + str(ACET) + ' -sol IPA:'+ str(IPA) + \
' -sol DMAD:' + str(DMAD) + ' -sol IPO:' + str(IPO) + \
' -sol PPN:' + str(PPN)

# create the topology file
with open('topol.top', 'w') as f_out:
    f_out.write('#include "_martini/martini_v3.0.0.itp"\n')
    f_out.write('#include "_martini/martini_v3.0.0_ions_v1.itp"\n')
    f_out.write('#include "_martini/martini_v3.0.0_probes_v1.itp"\n')
    f_out.write('#include "_martini/martini_v3.0.0_solvents_v1.itp"\n')
    f_out.write('#include "molecule_0.itp"\n')
    f_out.write('[ system ]\n')
    f_out.write('CG system for ' + pdb_file_name + '\n')
    f_out.write('[ molecules ]\n')
    f_out.write('molecule_0    1\n')

with open('prep.sh', 'w') as f_out:
    f_out.write('martinize2 -maxwarn 100 -f aa_fixed.pdb -x _cg.pdb -o _topol.top -scfix -cys auto  -elastic -p backbone -nt -merge ' + ChainID + ' -ss ' + ss + '\n')
    f_out.write('python insane_probes.py -f _cg.pdb -o _cg_sol.gro -p _insane.top -salt 0 -charge auto ' + sol_str + ' -pbc cubic -d 3.0' + '\n')
    f_out.write("sed -n -E '/^(CLBZ|PHEN|BENZ|ACE|IPA|DMAD|IPO|PPN|NA|CL|W)/p' _insane.top >> topol.top")

!chmod +x prep.sh
!./prep.sh
!sed -i '/^;/d' molecule_0.itp

In [ ]:
# set the path to gromacs gmx
gmx_path = '~/soft/gromacs-2022.3/bin/gmx'
# fix the number of threads and GPU ID if needed
with open('run.sh', 'w') as f_out:
    f_out.write(gmx_path + ' grompp -f _mdp/cg_em.mdp -c _cg_sol.gro -r _cg_sol.gro -p topol.top -o em.tpr -maxwarn 10\n')
    f_out.write(gmx_path + ' mdrun -deffnm em -nt 4 -gpu_id 0\n')
    f_out.write(gmx_path + ' grompp -f _mdp/cg_eq_1.mdp -c em.gro -r em.gro -p topol.top -o eq_1.tpr -maxwarn 10\n')
    f_out.write(gmx_path + ' mdrun -deffnm eq_1 -nt 4 -gpu_id 0\n')
    f_out.write(gmx_path + ' grompp -f _mdp/cg_eq_2.mdp -c eq_1.gro -r eq_1.gro -p topol.top -o eq_2.tpr -maxwarn 10\n')
    f_out.write(gmx_path + ' mdrun -deffnm eq_2 -nt 4 -gpu_id 0\n')
    f_out.write(gmx_path + ' grompp -f _mdp/cg_md.mdp -c eq_2.gro -r eq_2.gro -p topol.top -o prod.tpr -maxwarn 10\n')
    f_out.write(gmx_path + ' mdrun -deffnm prod -nt 4 -gpu_id 0\n')
    
!chmod +x run.sh
!./run.sh

In [ ]:
import MDAnalysis as mda
import MDAnalysis.transformations


# **Preprocess the trajectory**
# Firstly, each frame is translated to the protein's center of mass, 
# then solvent and probes are wraped around the protein, 
# and, finally, each frame is aligned to the protein again using the roto-translational fit.


u = mda.Universe('_cg_sol.gro', 'prod.xtc', in_memory=True)

# bonds in the coarse-grained model should be correctly set
# in order to unwrap the protein in case it's crossing the PBC
u_bonds = mda.Universe('_cg.pdb')
bonds = [(i[0].index, i[1].index) for i in u_bonds.bonds]
for trp in u_bonds.select_atoms('resname TRP').residues:
    bonds.append((trp.atoms[3].index, trp.atoms[4].index))
u.add_TopologyAttr('bonds', bonds)

prot = u.select_atoms('not resname W ION PHEN ACET IPA DMAD IPO PPN')
prot_BB = u.select_atoms('name BB and not resname W ION PHEN ACET IPA DMAD IPO PPN')
probes = u.select_atoms('resname PHEN ACET IPA DMAD IPO PPN')
sel_prot_probes = u.select_atoms('not resname W ION')
sel_all = u.select_atoms('all')

u_ref = mda.Universe("_cg_sol.gro")
prot_ref = u_ref.select_atoms('name BB and not resname W ION PHEN ACET IPA DMAD IPO PPN')

workflow = [mda.transformations.unwrap(prot),
            mda.transformations.fit_translation(prot_BB, prot_ref),
            mda.transformations.wrap(probes),
            mda.transformations.fit_rot_trans(prot_BB, prot_ref)]

u.trajectory.add_transformations(*workflow)

sel_prot_probes.write("prod_wraped.pdb")

with mda.Writer("prod_wraped.xtc", sel_prot_probes.n_atoms) as W:
    for ts in u.trajectory:
        W.write(sel_prot_probes)

u.trajectory[-1]
sel_all.write('prod_last.gro')
print('Done.')

In [ ]:
#@title **Calculate densities**
from MDAnalysis.analysis import align
from MDAnalysis.analysis.density import DensityAnalysis
from gridData import Grid
import numpy as np

# bandwidth for MeanShift
bandwidth = 6.0
# temperature (should match the temperature in gromacs mdp file)
temp = 303.15
# calculate bulk densities
u = mda.Universe('prod_last.gro')
n_water = u.select_atoms('resname W').n_atoms * 4

probes = []
probe_resnames = ['PHEN', 'ACET', 'IPA', 'DMAD', 'IPO', 'PPN']
probe_names = ['PHE', 'ACT', 'IPA', 'DMA', 'IPL', 'ACN']

n_probe = u.select_atoms('not name VS* and (resname ' + ' '.join(probe_resnames) + ')', updating=True).n_atoms
n0 = n_probe/(u.dimensions[0]*u.dimensions[1]*u.dimensions[2])
# uncomment to calculate n0 as in 10.1073/pnas.2214024119
#n0 = 6.02e23*55.56*n_probe/n_water/10e27
probes.append([probe_resnames, 'ALL', n0])

for p, n in zip(probe_resnames, probe_names):
    n_probe = u.select_atoms('not name VS* and (resname ' + p + ')', updating=True).n_atoms
    n0 = n_probe/(u.dimensions[0]*u.dimensions[1]*u.dimensions[2])
    #n0 = 6.02e23*55.56*n_probe/n_water/10e27
    probes.append([[p], n, n0])

grid_d = 2 # grid spacing
kt = 0.0083188*temp # kT value in kJ/mol for cut-off

# align the AA model to the CG model

ref = mda.Universe('prod_wraped.pdb')
ref.select_atoms('name BB').names = 'CA'
mobile = mda.Universe('aa_fixed.pdb')

align.alignto(mobile, ref, select='name CA and not altloc B C D E', match_atoms=False)
mobile.select_atoms('protein').write('aa_aligned.pdb')

u = mda.Universe('prod_wraped.pdb', 'prod_wraped.xtc')

protein = u.select_atoms('protein')
protein_COM = protein.center_of_mass()

for p in probes:
    # density calculation
    p_name = p[0]
    out_name = p[1]
    n0 = p[2]

    if n0 == 0:
        continue

    p_sel = u.select_atoms('not name VS* and (resname ' + ' '.join(p_name) + ' and around 6 protein)', updating=True)

    D = DensityAnalysis(p_sel, delta=grid_d, gridcenter=protein_COM, \
                        xdim=u.dimensions[0], ydim=u.dimensions[1], zdim=u.dimensions[2])
    D.run()

    grid = D.results.density.grid
    grid = np.log(grid/n0)
    grid[np.isneginf(grid)] = 0 # remove inf due to log(0)
    grid = (-8.314*temp/1000)*grid # density -> kJ/mol
    grid = grid*(grid < -1*kt)
    D.results.density.grid = grid
    D.results.density.export('dens_' + out_name + '.dx')
    print("Wrote file for: {}, min value: {:.2f}".format(out_name, grid.min()))
print('Done.')

In [ ]:
from sklearn.cluster import MeanShift

# **Density clustering**

# Change the bandwidth parameter if you are not satisfied with clustering.
# Check for more details, https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html
# Set to 0 for automatic estimation, however, 6 usually performs good.

bandwidth = 6

def pdb_line(at_num, at_type, res_type, chain_id, res_num, xyz, occ, temp):
    # returns a pdb-formatted line
    return "%6s%5s %4s %3s %1s%4d    %8.3f%8.3f%8.3f%6.2f%6.2f" \
            % ("ATOM  ", at_num, at_type, res_type, chain_id, res_num, xyz[0], xyz[1], xyz[2], occ, temp)

f_out_centers_mean_ene = open('centers_mean_ene.pdb', 'w')
f_out_centers_min_ene = open('centers_min_ene.pdb', 'w')
f_out_centers_sum_ene = open('centers_sum_ene.pdb', 'w')
f_out_clusters_mean_ene = open('clusters_mean_ene.pdb', 'w')
f_out_clusters_min_ene = open('clusters_min_ene.pdb', 'w')
f_out_clusters_sum_ene = open('clusters_sum_ene.pdb', 'w')

for p in probes:
    p_name = p[0]
    out_name = p[1]
    n0 = p[2]

    if n0 == 0:
        continue

    G = Grid('dens_' + out_name + '.dx')

    grid = G.grid
    edges = G.edges
    hotspots, energies = [], []

    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            for k in range(grid.shape[2]):
                if grid[i,j,k] < 0:
                    hotspots.append([edges[0][i], edges[1][j], edges[2][k]])
                    energies.append(grid[i,j,k])

    hotspots = np.array(hotspots)
    energies = np.array(energies)
    if bandwidth != 0:
        clustering = MeanShift(bandwidth=bandwidth).fit(hotspots)
    else:
        clustering = MeanShift().fit(hotspots)

    clusters = {}
    for i in np.unique(clustering.labels_):
        if energies[clustering.labels_ == i].sum() < -10:
            clusters[i] = [hotspots[clustering.labels_ == i].shape[0], \
                   energies[clustering.labels_ == i].mean(), \
                   energies[clustering.labels_ == i].min(), \
                   energies[clustering.labels_ == i].sum(), \
                   hotspots[clustering.labels_ == i][energies[clustering.labels_ == i].argmin()], \
                   hotspots[clustering.labels_ == i].mean(axis = 0)]

    # clusters[id] = [size, mean energy, min energy, sum energy, [position of min energy], [center of cluster]]
    clusters_mean_sort = [i[0] for i in sorted(clusters.items(), \
                         key=lambda item: item[1][1], reverse=False)]
    clusters_min_sort = [i[0] for i in sorted(clusters.items(), \
                         key=lambda item: item[1][2], reverse=False)]
    clusters_sum_sort = [i[0] for i in sorted(clusters.items(), \
                          key=lambda item: item[1][3], reverse=False)]

    # pdb_line(at_num, at_type, res_type, chain_id, res_num, x, y, z, occ, temp)
    # writing centers of clusters, sorted by mean energy
    for i, j in enumerate(clusters_mean_sort):
        f_out_centers_mean_ene.write(pdb_line(i + 1, out_name, out_name, 'A', i + 1, clusters[j][4], clusters[j][2], clusters[j][1]) + '\n')
    # writing centers of clusters, sorted by min energy
    for i, j in enumerate(clusters_min_sort):
        f_out_centers_min_ene.write(pdb_line(i + 1, out_name, out_name, 'A', i + 1, clusters[j][4], clusters[j][1], clusters[j][2]) + '\n')
    # writing centers of clusters, sorted by sum energy
    for i, j in enumerate(clusters_sum_sort):
        f_out_centers_sum_ene.write(pdb_line(i + 1, out_name, out_name, 'A', i + 1, clusters[j][5], clusters[j][1], clusters[j][3]) + '\n')
    # writing clusters, sorted by mean energy
    k = 1
    for i, j in enumerate(clusters_mean_sort):
        xyz = hotspots[clustering.labels_ == j]
        ene = energies[clustering.labels_ == j]
        for spot in zip(ene, xyz):
            f_out_clusters_mean_ene.write(pdb_line(k, out_name, out_name, 'A', i + 1, spot[1], spot[0], clusters[j][1]) + '\n')
            k += 1

    # writing clusters, sorted by min energy
    k = 1
    for i, j in enumerate(clusters_min_sort):
        xyz = hotspots[clustering.labels_ == j]
        ene = energies[clustering.labels_ == j]
        for spot in zip(ene, xyz):
            f_out_clusters_min_ene.write(pdb_line(k, out_name, out_name, 'A', i + 1, spot[1], spot[0], clusters[j][2]) + '\n')
            k += 1

    # writing clusters, sorted by sum energy
    k = 1
    for i, j in enumerate(clusters_sum_sort):
        xyz = hotspots[clustering.labels_ == j]
        ene = energies[clustering.labels_ == j]
        for spot in zip(ene, xyz):
            f_out_clusters_sum_ene.write(pdb_line(k, out_name, out_name, 'A', i + 1, spot[1], spot[0], clusters[j][3]) + '\n')
            k += 1

f_out_centers_mean_ene.close()
f_out_centers_min_ene.close()
f_out_centers_sum_ene.close()
f_out_clusters_mean_ene.close()
f_out_clusters_min_ene.close()
f_out_clusters_sum_ene.close()
print('Done.')